<a href="https://colab.research.google.com/github/sbooeshaghi/azucar/blob/main/analysis/293T/obs2/imports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet -U upsetplot scikit-learn git+http://github.com/dirguis/ipfn@master

In [2]:
!git clone https://github.com/sbooeshaghi/azucar.git

Cloning into 'azucar'...
remote: Enumerating objects: 1571, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 1571 (delta 190), reused 50 (delta 14), pack-reused 1242
Receiving objects: 100% (1571/1571), 1.70 GiB | 28.00 MiB/s, done.
Resolving deltas: 100% (666/666), done.
Checking out files: 100% (293/293), done.


In [1]:
import os
from scipy.io import mmread
import pandas as pd
import numpy as np
from collections import defaultdict

In [22]:
sample = "293T"
observation = "obs2"

base_data = f"azucar/analysis/{sample}/{observation}/out"
base_mark = f"azucar/analysis/{sample}/{observation}/assign"

matrix_fn  = os.path.join(base_data, "matrix.mtx")
genes_fn   = os.path.join(base_data, "genes.txt")
barcodes_fn   = os.path.join(base_data, "barcodes.txt")

!gunzip $base_data/*.gz

gzip: azucar/analysis/293T/obs2/out/*.gz: No such file or directory


In [23]:
!rm *.mtx *.txt *.ec

In [44]:
from azucar.analysis.scripts.mx_sanitize import mx_sanitize
from azucar.analysis.scripts.mx_filter import mx_filter, knee, gmm
from azucar.analysis.scripts.mx_norm import mx_norm
from azucar.analysis.scripts.ec_index import ec_index
from azucar.analysis.scripts.ec_matrix import ec_matrix
from azucar.analysis.scripts.mx_select import mx_select
from azucar.analysis.scripts.mx_extract import mx_extract
from azucar.analysis.scripts.mx_assign import mx_assign
from azucar.analysis.scripts.mx_diff import mx_diff

from azucar.analysis.scripts.utils import read_str_list, read_markers_str, get_marker_centroids, read_markers_ec, write_markers

In [25]:
markers_fn = os.path.join(base_mark, "markers.txt")

In [28]:
# drop barcodes and genes that sum to zero, update barcodes and genes file
mx_sanitize(matrix_fn, barcodes_fn, genes_fn, 
            "./san.matrix.mtx", 
            "./san.barcodes.txt", 
            "./san.genes.txt")

# knee plot gmm filter
mx_filter("./san.matrix.mtx",
          "./san.barcodes.txt",
          "./san.fil.matrix.mtx", 
          "./san.fil.barcodes.txt",
          comps=[3,3])#, select_axis=2)

dbco = mmread("./san.fil.matrix.mtx").toarray()[:,-1]

# normalize matrix (log1p -> ipf)
mx_norm("./san.fil.matrix.mtx", 
        "./san.log1p.matrix.mtx", how="log1p")

mx_norm("./san.log1p.matrix.mtx", 
        "./san.norm.matrix.mtx", how="ipf")#, target_sum=1_000_000)

dbco_norm = mmread("./san.norm.matrix.mtx").toarray()[:,-1]

# index the markers -> markers.ec marker_genes.txt groups.txt
ec_index(markers_fn, 
        "./markers.ec", 
        "./groups.txt", 
        "./marker_genes.txt")
# get the gene ids -> select.txt (selects in order of markers.ec)
mx_select(markers_fn, 
          "./san.genes.txt", 
          "./select.txt")
# extract elements from matrix that are of interest, rows / columns (with associated metadata)
mx_extract("san.norm.matrix.mtx", 
          "san.genes.txt", 
          "select.txt", 
          "san.norm.extr.matrix.mtx", 
          "san.extr.genes.txt", axis=1)

# perform assignments with markers and matrices
mx_assign("san.norm.extr.matrix.mtx", 
          "san.fil.barcodes.txt", 
          "san.extr.genes.txt", 
          "markers.ec", 
          "groups.txt",
          "assignments.txt")

Filtered to 2,514 cells with at least 137 UMIs.
Initialization 0
  Iteration 10	 time lapse 0.02031s	 ll change 0.04029
  Iteration 20	 time lapse 0.01897s	 ll change 0.00013
Initialization converged: True	 time lapse 0.04205s	 ll -7.84612


In [29]:
df = pd.read_csv("assignments.txt", sep="\t", index_col=0)
print("shape: ", df.shape)

# original counts (this is the same as make df above)
raw_mtx = mmread("./san.fil.matrix.mtx").toarray()
raw_genes = []
read_str_list("./san.genes.txt", raw_genes)
for idx, v in enumerate(raw_mtx.T):
    df[f"{raw_genes[idx]}"] = v # np.exp(v) - 1

df["dbco"] = dbco
df["dbco_ipf"] = dbco_norm

# for the np log
df.loc[df['ent'] == 0.0, 'ent'] = np.finfo(float).tiny

## find entropy cutoff
adj_ent = -np.log(df["ent"].values)

u, xx, v = knee(np.log1p(adj_ent.reshape(-1,1)), sum_axis=1)
(cutoff, ncells) = gmm(xx, v, comps=[3])
cutoff = np.exp(cutoff) - 1 # adjust (since we log1p it)
ent_cutoff = np.exp(-cutoff)

print(ncells, ent_cutoff)

shape:  (2514, 10)
511 5.5137080071387077e-17


In [41]:
df.query(f"ent < {ent_cutoff}").groupby("label").agg({
      "mtag1_ipf": "mean", 
      "mtag2_ipf": "mean",
      "mtag3_ipf": "mean",
      "mtag4_ipf": "mean",
      "dbco_ipf": "mean",
      "label": "count"
      }).astype(float)

,mtag1_ipf,mtag2_ipf,mtag3_ipf,mtag4_ipf,dbco_ipf,label
label,,,,,,
control,0.097275,0.094250,0.313937,5.730803,8.385718,168.0
no_sugar,13.366825,0.073267,0.841018,0.295870,0.044878,204.0
tmg,0.202255,12.637238,1.171351,0.492672,0.118505,140.0


In [31]:
# fix mx_diff to save file
mdf = mx_diff("san.norm.extr.matrix.mtx", 
              "san.barcodes.txt", 
              "san.extr.genes.txt", 
              df["label"].values)

def convert_mtx(*x):
  # x is a pandas series with multiple values
  # but because our matrix has unique group, tag pairs
  # there is only one element in the series
  return x[0].values[0]

es = mdf.groupby(["group_id", "name"])["es"].apply(convert_mtx).unstack()
pv = mdf.groupby(["group_id", "name"])["p_corr"].apply(convert_mtx).unstack()

xa, ya = np.where(es.values > 0)
check = defaultdict(list)
conditions = es.index.values
tags = es.columns.values
for i, j in zip(xa, ya):
  check[conditions[i]].append(tags[j])

markers = defaultdict(list)
read_markers_str(markers_fn, markers)

# number in true only, number in intersection, number in check only
for ct, gs in markers.items():
  tm = markers[ct]
  cm = check.get(ct, [])

  left  = np.setdiff1d  (tm, cm).shape[0]
  itx   = np.intersect1d(tm, cm).shape[0]
  right = np.setdiff1d  (cm, tm).shape[0]

  print(f"{left, itx, right} \t {ct} \t {tm} ∩ {cm}")

15-Feb-22 19:56:31 - 1 of 3 assignments: control
15-Feb-22 19:56:31 - 2 of 3 assignments: no_sugar
15-Feb-22 19:56:31 - 3 of 3 assignments: tmg


(0, 2, 0) 	 no_sugar 	 ['mtag1', 'mtag3'] ∩ ['mtag1', 'mtag3']
(1, 1, 0) 	 control 	 ['mtag3', 'mtag4'] ∩ ['mtag4']
(0, 2, 2) 	 tmg 	 ['mtag2', 'mtag3'] ∩ ['mtag1', 'mtag2', 'mtag3', 'mtag4']


In [32]:
(es.values > 0).astype(int)

array([[0, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 1, 1, 1]])

In [33]:
es

name,mtag1,mtag2,mtag3,mtag4
group_id,,,,
control,-0.431252,-0.220084,-0.243212,0.518050
no_sugar,0.390362,NaN,0.132612,NaN
tmg,0.134967,0.844859,0.131266,0.191005


In [34]:
pv

name,mtag1,mtag2,mtag3,mtag4
group_id,,,,
control,7.520579e-58,9.277857e-19,1.778996e-17,3.015418e-67
no_sugar,1.900017e-11,NaN,1.039295e-02,NaN
tmg,3.761492e-06,3.343428e-268,1.558543e-06,4.879358e-13


In [35]:
ec_matrix("markers.ec", "markers.mtx")

In [36]:
!cat markers.mtx

%%MatrixMarket matrix coordinate integer general
%
3 4 6
1 1 1
1 2 1
2 2 1
2 3 1
3 2 1
3 4 1


In [37]:
markers

defaultdict(list,
            {'control': ['mtag3', 'mtag4'],
             'no_sugar': ['mtag1', 'mtag3'],
             'tmg': ['mtag2', 'mtag3']})

In [38]:
# 0: no_sugar
# 1: control
# 2: tmg

In [43]:
df.query(f"ent < {ent_cutoff}").sort_values("label").query("label == 'tmg'").filter(like="_ipf")

,mtag1_ipf,mtag3_ipf,mtag4_ipf,mtag2_ipf,dbco_ipf
TGAATGCAGAGCCGTA,0.000000,1.605603,0.000000,13.016421,0.000000
ACACGCGGTGCCCAGT,1.557726,1.631942,1.878657,9.553671,0.000000
CGTTGGGGTGCCTACG,0.000000,0.000000,0.000000,14.622033,0.000000
CGTTGGGGTGCCTTTC,0.000000,1.584408,0.000000,13.037617,0.000000
TGAGCGCTCCACACCT,0.000000,0.000000,0.000000,14.622033,0.000000
...,...,...,...,...,...
CTGCTCAGTTTCAGAC,0.000000,3.888125,2.050677,8.683201,0.000000
TCAGGGCTCCGCATAA,0.964877,1.010847,2.327333,10.318946,0.000000
CTGCTCAGTTGGGAAC,1.325843,3.242947,1.599000,7.570383,0.883816
CAGGGCTAGGCACTCC,0.000000,1.533861,0.000000,13.088164,0.000000


In [ ]:
# tags
# AAGGCAGACG	mtag1
# GGCTGCGCAC	mtag2
# TAGTTGACAT	mtag3
# GACGCGCGTT	mtag4
# AGCGAACCGC	dbco

# mtag4 has 8bp shared seq with mtag4_rc

# tags_revc
# CGTCTGCCTT mtag1
# GTGCGCAGCC mtag2
# ATGTCAACTA mtag3
# AACGCGCGTC mtag4
# GCGGTTCGCT dbco

In [66]:
def merge(ecs1, ecs2):
  m = defaultdict(set)

  uniq = set(list(ecs1.keys()) + list(ecs2.keys()))
  for k in uniq:
    m[k].update(ecs2.get(k, []))
    m[k].update(ecs1.get(k, []))

  # convert from set to list
  m = dict(zip(m.keys(), map(list, m.values())))
  return m

def ec_merge(ec_a_fn, ec_b_fn, ec_merge_fn):
  ec_a = defaultdict(list)
  read_markers_str(ec_a_fn, ec_a)

  ec_b = defaultdict(list)
  read_markers_str(ec_b_fn, ec_b)

  m = merge(ec_a, ec_b)

  write_markers(ec_merge_fn, m)

In [61]:
markers_a = {
    "a": [1,2,3],
    "b": [2,4],
    "c": [1]
}

markers_b = {
    "a": [4],
    "b": [1],
    "c": [2,3],
    "d": [2]
}

In [62]:
merge(markers_a, markers_b)

{'a': [1, 2, 3, 4], 'b': [1, 2, 4], 'c': [1, 2, 3], 'd': [2]}

In [64]:
write_markers("test_a.txt", markers_a)
write_markers("test_b.txt", markers_b)

In [67]:
ec_merge("test_a.txt", "test_b.txt", "merge.txt")

In [68]:
!cat merge.txt

d	2
a	2,1,4,3
b	2,1,4
c	2,1,3


## Manual Assignment

In [14]:
## John Assignment

# load data
mtx = mmread("./san.matrix.mtx").toarray()

bcs = []
read_str_list("./san.barcodes.txt", bcs)

genes = []
read_str_list("./san.genes.txt", genes)

# make df
m = pd.DataFrame(mtx, index = bcs, columns=genes)

# John assignment preprocessing
john = m.div(m.sum(axis=0), axis=1) # normalize gene to be same
john["mtag3"] = john["mtag3"] * 3.
john *= 100000
john.head()

# assignment procedure
def john_assign(row):
  thresh = 40

  # try all conditions
  # where tags for conditions are high
  # and non-tags are low
  # assign
  label = []
  for c, v in markers.items():
    other = list(set(genes) - set(v))

    other.remove("dbco")
    c1 = row[v[0]] > thresh
    c2 = row[v[1]] > thresh
    c3 = row[other[0]] < thresh
    c4 = row[other[1]] < thresh

    if (c1&c2) & (c3&c4):
      label.append(c)
  return label

blah = john.apply(john_assign, axis=1)
d = blah[blah.map(lambda d: len(d) > 0)]
d = d.map(lambda x: x[0])
john["label"] = john.index.map(d)
john = john.dropna()

In [15]:
## Top two assignment

# load in data
mtx = mmread("./san.matrix.mtx").toarray()[:,:-1]
dbco = mmread("./san.matrix.mtx").toarray()[:,-1]

bcs = []
read_str_list("./san.barcodes.txt", bcs)

genes = []
read_str_list("./san.genes.txt", genes)
genes.remove("dbco")

# make df
man = pd.DataFrame(mtx, index=bcs, columns=genes)

# pick top two expressed barcodes and assign based on map
bcmap = {
    (1,3): "no_sugar",
    (3,4): "control",
    (2,3): "tmg"
}
ind = np.sort(np.argpartition(mtx, -2)[:,-2:], axis=1) + 1
assn = [bcmap.get(tuple(i), None) for i in ind]
man["dbco"] = dbco
man["label"] = assn

In [16]:
# gmm v john
common = np.intersect1d(john.index.values, df.index.values)
print((df.loc[common].label.values == john.loc[common].label.values).sum(), "out of", common.shape[0])

261 out of 454


In [17]:
# man v john
common = np.intersect1d(john.index.values, man.index.values)
print((man.loc[common].label.values == john.loc[common].label.values).sum(), "out of", common.shape[0])

446 out of 454


In [18]:
# man v df
common = np.intersect1d(df.index.values, man.index.values)
print((man.loc[common].label.values == df.loc[common].label.values).sum(), "out of", common.shape[0])

1181 out of 1756


In [19]:
john.groupby("label")["dbco"].mean()


label
control     25.372801
no_sugar    17.837735
tmg         40.977006
Name: dbco, dtype: float64

In [20]:
man.loc[common].groupby("label")["dbco"].mean()

label
control     19.606828
no_sugar    32.100186
tmg         67.598039
Name: dbco, dtype: float64

In [21]:
df.loc[common].groupby("label")["dbco"].mean()

label
control     24.150649
no_sugar    49.059133
tmg          0.026667
Name: dbco, dtype: float64